In [1]:
import json
import pandas as pd
import geopandas as gpd
import sqlite3
import os

# Paths to shapefiles and JSON files
house_geo_path = r"C:\Users\CrockfordSam\OneDrive - Maryland Legal Aid\Desktop\Data Projects\StateLegInformer\shapes\houseGeo"
senate_geo_path = r"C:\Users\CrockfordSam\OneDrive - Maryland Legal Aid\Desktop\Data Projects\StateLegInformer\shapes\senGeo"
house_json_path = r"maryland_house_members.json"
senate_json_path = r"maryland_senate_members.json"
db_name = "districts.db"

# Function to normalize JSON data into a DataFrame with chamber
def json_to_dataframe(json_data, chamber):
    df = pd.json_normalize(json_data)
    df.columns = [col.replace('data.', '') for col in df.columns]
    df['chamber'] = chamber
    return df

# Function to load shapefiles into GeoDataFrames
def load_shapefiles(path):
    shapefiles = [os.path.join(path, file) for file in os.listdir(path) if file.endswith('.shp')]
    gdfs = [gpd.read_file(shp) for shp in shapefiles]
    return pd.concat(gdfs, ignore_index=True)

# Function to normalize district names
def normalize_districts(df, column_name):
    return (
        df[column_name]
        .str.strip()
        .str.upper()
        .str.replace('SD', '', regex=False)  # Remove 'SD' prefix for Senate districts
        .str.lstrip('0')  # Remove leading zeros
    )

# Function to save the combined DataFrame with geometry to SQLite
def save_to_sqlite(gdf, db_name):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    # Create table with all attributes and geometry
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS districts (
            id INTEGER PRIMARY KEY,
            name TEXT,
            chamber TEXT,
            district TEXT,
            party TEXT,
            geometry TEXT,
            committee TEXT, 
            notes TEXT
        )
    ''')

    for _, row in gdf.iterrows():
        name = row.get('name')
        chamber = row.get('chamber')  # After merge, chambers may be suffixed
        district = row.get('District')
        party = row.get('Party')
        committee = row.get('Committee Assignment(s)')
        geometry = row['geometry'].wkt if row['geometry'] else None
        notes = None 

        cursor.execute('''
            INSERT INTO districts (name, chamber, district, party, geometry, committee,notes)
            VALUES (?, ?, ?, ?, ?, ?, ?)
        ''', (name, chamber, district, party, geometry, committee, notes))

    conn.commit()
    conn.close()

# Load JSON data
with open(house_json_path, "r") as house_file:
    json_house = json.load(house_file)
with open(senate_json_path, "r") as senate_file:
    json_senate = json.load(senate_file)

# Convert JSON data to DataFrames
df_house = json_to_dataframe(json_house, "House")
df_senate = json_to_dataframe(json_senate, "Senate")

# Combine the JSON DataFrames
df_combined = pd.concat([df_house, df_senate], ignore_index=True)

# Normalize districts in combined DataFrame
df_combined['District'] = normalize_districts(df_combined, 'District')

# Load shapefiles into GeoDataFrames
gdf_house = load_shapefiles(house_geo_path)
gdf_senate = load_shapefiles(senate_geo_path)

# Add chamber column to GeoDataFrames
gdf_house['chamber'] = "House"
gdf_senate['chamber'] = "Senate"

# Combine the GeoDataFrames
gdf_combined_geo = pd.concat([gdf_house, gdf_senate], ignore_index=True)

# Normalize districts in GeoDataFrame
gdf_combined_geo['DISTRICT'] = normalize_districts(gdf_combined_geo, 'DISTRICT')

# Merge combined DataFrame with GeoDataFrame on District and chamber
gdf_final = pd.merge(
    df_combined,
    gdf_combined_geo,
    left_on=["District", "chamber"],
    right_on=["DISTRICT", "chamber"],
    how="left"
)

# Convert to GeoDataFrame
gdf_final = gpd.GeoDataFrame(gdf_final, geometry=gdf_final['geometry'])

# Check for missing geometries
missing_geo = gdf_final[gdf_final['geometry'].isnull()]
if not missing_geo.empty:
    print("Missing geometries for the following districts:")
    print(missing_geo[['District', 'chamber']])

# Save to SQLite
save_to_sqlite(gdf_final, db_name)

print("Data successfully saved to SQLite.")



Missing geometries for the following districts:
    District chamber
139      NaN   House
185      NaN  Senate
186      NaN  Senate
Data successfully saved to SQLite.
